In [1]:
!pip install transformers

You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [14]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name_or_path = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [3]:
text = "Вопрос: 'Сколько будет 2+2?'\nОтвет:"
input_ids = tokenizer.encode(text, return_tensors='pt').to(DEVICE)
out = model.generate(input_ids, do_sample=False)

generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Вопрос: 'Сколько будет 2+2?'
Ответ: '2+2=4'


In [4]:
text = 'Токенизируй меня'
tokens = tokenizer.encode(text, add_special_tokens=False)
decoded_tokens = [tokenizer.decode([token]) for token in tokens]

print('text: ', text)
print('tokens: ', tokens)
print('decoded tokens: ', decoded_tokens)

text:  Токенизируй меня
tokens:  [789, 368, 337, 848, 28306, 703]
decoded tokens:  ['Т', 'ок', 'ени', 'зи', 'руй', ' меня']


In [5]:
text = 'Определение: "Нейронная сеть" - это'
input_ids = tokenizer.encode(text, return_tensors='pt').to(DEVICE)

In [6]:
out = model.generate(input_ids,
                     do_sample=True,
                     top_k=20,
                     top_p=0.8,
                     max_length=30,
                     temperature=0.8
                    )
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Определение: "Нейронная сеть" - это... Что такое "Нейронная сеть"?<s>
Нейронная сеть —


In [15]:
from transformers import TextDataset, DataCollatorForLanguageModeling

In [16]:
train_path = 'train_dataset.txt'
    
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

c:\users\admin\appdata\local\programs\python\python39\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [18]:
from transformers import Trainer, TrainingArguments

In [19]:
training_args = TrainingArguments(
    output_dir='./finetuned',
    overwrite_output_dir=True,
    num_train_epochs=200,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=10,
    gradient_accumulation_steps=16,
#     no_cuda=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers=(torch.optim.AdamW(model.parameters(), lr=1e-5), None),
)

In [20]:
trainer.train()

***** Running training *****
  Num examples = 5
  Num Epochs = 200
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 200


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=200, training_loss=0.02246487855911255, metrics={'train_runtime': 49.4721, 'train_samples_per_second': 20.213, 'train_steps_per_second': 4.043, 'total_flos': 32661504000000.0, 'train_loss': 0.02246487855911255, 'epoch': 200.0})

In [21]:
model.save_pretrained('saved_model_small')

Configuration saved in saved_model_small\config.json
Model weights saved in saved_model_small\pytorch_model.bin


In [22]:
model = GPT2LMHeadModel.from_pretrained('saved_model_small').to(DEVICE)

loading configuration file saved_model_small\config.json
Model config GPT2Config {
  "_name_or_path": "sberbank-ai/rugpt3small_based_on_gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "use_cache": true,
  "vocab_size": 50264
}

loading weights file saved_model_small\pytorch_model.bi

In [23]:
text = 'Рыбный угар |0| '
input_ids = tokenizer.encode(text, return_tensors='pt').to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        num_return_sequences=1,
                        no_repeat_ngram_size=3,
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=500,                     
                        eos_token_id=tokenizer.eos_token_id,
                        pad_token_id=tokenizer.pad_token_id,
                        remove_invalid_values=True,
                        )

generated_text = list(map(tokenizer.decode, out))[0]

print(generated_text.replace('|0|', '\n').split('<s>')[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Рыбный угар 
 
Важное сладкое блюдо советской и постсоветской истории. Легкое, пышное тесто, максимум яблочной начинки — у шарлотки всегда был образ приятного, простого и при этом лакомого и диетического блюда. Яблоки настоятельно рекомендуем взять из кислых сортов — вроде антоновки. Их можно класть как сырыми, так и предварительно слегка карамелизованными на сковородке. И сахара лучше не жалеть. Он магическим образом распределяется в нужном количестве в тесте, а излишки образуют сладкую корочку.
1. Классическая шарлотка.
Углекислота нужна для производства желатина. Его можно получить из молока, творога, кефира. Все они образуют одинаковую консистенцию.
Желтки взбить с сахаром, добавить муку.
3. Классическое шарлотковое тесто.
Тесто по консистенции должно напоминать сметану. Вылить половину теста на разогретую сковороду, выложить половину второй половиной теста, разложить равномерно половину второй половины теста, залить второй половиной первого теста, накрыть второй половиной первой п